In [64]:
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif, chi2

In [65]:
#import Data

#import PL data
PL_18_19 = pd.read_csv('Data/other_years/E_18_19.csv')
PL_19_20 = pd.read_csv('Data/other_years/E_19_20.csv')
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_21_22 = pd.read_csv('Data/other_years/E_21_22.csv')

#import xG for PL
PL_18_19_xG = pd.read_csv('Data/other_years/E_18_19_xG.csv')
PL_19_20_xG = pd.read_csv('Data/other_years/E_19_20_xG.csv')
PL_20_21_xG = pd.read_csv('Data/2020-21/Fixtures/E_20_21_xG.csv')
PL_21_22_xG = pd.read_csv('Data/other_years/E_21_22_xG.csv')

lineups = pd.read_csv('Data/lineups.csv')
lineups


C:\Users\giuli\AppData\Local\Temp\ipykernel_79800\2504626855.py:15: DtypeWarning: Columns (136,141,143,145,146,148,149,151,152,154) have mixed types. Specify dtype option on import or set low_memory=False.
  lineups = pd.read_csv('Data/lineups.csv')


,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_bench_10,home_bench_10_minute,home_bench_11_num,home_bench_11,home_bench_11_minute,away_bench_10_num,away_bench_10,away_bench_10_minute,away_starting_14_num,away_starting_14
0,18123,Charlton Athletic,Everton,"Saturday, August 18",2001,14:00,"20,451","The Valley, London, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18124,Derby County,Blackburn Rovers,"Saturday, August 18",2001,14:00,"28,236","Pride Park Stadium, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18125,Leeds United,Southampton,"Saturday, August 18",2001,14:00,"39,715","Elland Road, Leeds, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18126,Leicester City,Bolton Wanderers,"Saturday, August 18",2001,14:00,"19,987","Filbert Street, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18127,Liverpool,West Ham United,"Saturday, August 18",2001,14:00,"43,935","Anfield, Liverpool, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7938,605704,Brighton & Hove Albion,Southampton,"Sunday, April 24",2021,13:00,NaN,"Amex Stadium, Falmer, England",2021-22 English Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7939,605708,Burnley,Wolverhampton Wanderers,"Sunday, April 24",2021,13:00,NaN,"Turf Moor, Burnley, England",2021-22 English Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7940,605707,Chelsea,West Ham United,"Sunday, April 24",2021,13:00,"32,231","Stamford Bridge, London, England",2021-22 English Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7941,605700,Liverpool,Everton,"Sunday, April 24",2021,15:30,NaN,"Anfield, Liverpool, England",2021-22 English Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#Merge data
PL_total = pd.concat([PL_18_19, PL_19_20, PL_20_21, PL_21_22])
PL_total_xG = pd.concat([PL_18_19_xG, PL_19_20_xG, PL_20_21_xG, PL_21_22_xG])

#remove unnecessary columns
not_relevant_columns = ['BbOU','BbMx>2.5','BbAv>2.5','BbMx<2.5','B365>2.5','B365<2.5','P>2.5','P<2.5','Max>2.5','Max<2.5','Avg>2.5','Avg<2.5','BbAH','BbAHh','AHh','BbMxAHH','BbAvAHH','BbMxAHA','BbAvAHA','B365AHH','B365AHA','PAHH','PAHA','MaxAHH','MaxAHA','AvgAHH','AvgAHA','Time','MaxH','MaxD','MaxA','AvgH','AvgD','AvgA','B365CH','B365CD','B365CA','BWCH','BWCD','BWCA','IWCH','IWCD','IWCA','WHCH','WHCD','WHCA','VCCH','VCCD','VCCA','MaxCH','MaxCD','MaxCA','AvgCH','AvgCA','AvgCD','B365C>2.5','B365C<2.5','PC>2.5','PC<2.5','AHCh','B365CAHH','B365CAHA','PCAHH','PCAHA','MaxCAHH','MaxCAHA','AvgCAHH','AvgCAHA','BbAv<2.5','MaxC>2.5','MaxC<2.5','AvgC>2.5','AvgC<2.5']
#reset index
PL_total.drop( axis= 1, labels=not_relevant_columns, inplace=True)
PL_total.reset_index(inplace=False, drop=True)
PL_total['Date'] = pd.to_datetime(PL_total['Date'], dayfirst=True)
#update team names for merging
PL_total.replace(['Man City', 'Man United', 'Cardiff', 'Leeds', 'Leicester','Newcastle', 'Norwich','Sheffield United'], ['Manchester City', 'Manchester Utd', 'Cardiff City', 'Leeds United', 'Leicester City','Newcastle Utd','Norwich City','Sheffield Utd'], inplace=True)

PL_20_21.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester Utd', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)

#remove nan values and reset index
PL_total_xG.drop(labels='Notes', axis=1, inplace=True)
PL_total_xG.dropna(axis=0, inplace=True, how='all')
PL_total_xG.reset_index(inplace=True, drop=True)

#create ID's for join
PL_total['ID'] = PL_total['Date'].astype(str) + PL_total ['HomeTeam'] + PL_total ['AwayTeam']
PL_total_xG['ID'] = PL_total_xG['Date'].astype(str) + PL_total_xG ['Home'] + PL_total_xG ['Away']

#merge dataframes
PL = pd.merge(PL_total, PL_total_xG, on='ID', how='left')

PL


,Div,Date_x,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee_y,Match Report
0,E0,2018-08-10,Manchester Utd,Leicester City,2,1,H,1,0,H,...,20:00 (21:00),Manchester Utd,1.7,2–1,1.0,Leicester City,74439.0,Old Trafford,Andre Marriner,Match Report
1,E0,2018-08-11,Bournemouth,Cardiff City,2,0,H,1,0,H,...,15:00 (16:00),Bournemouth,2.1,2–0,1.2,Cardiff City,10353.0,Vitality Stadium,Kevin Friend,Match Report
2,E0,2018-08-11,Fulham,Crystal Palace,0,2,A,0,1,A,...,15:00 (16:00),Fulham,0.6,0–2,1.0,Crystal Palace,24821.0,Craven Cottage,Mike Dean,Match Report
3,E0,2018-08-11,Huddersfield,Chelsea,0,3,A,0,2,A,...,15:00 (16:00),Huddersfield,0.3,0–3,2.0,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,Match Report
4,E0,2018-08-11,Newcastle Utd,Tottenham,1,2,A,1,2,A,...,12:30 (13:30),Newcastle Utd,0.9,1–2,2.0,Tottenham,51749.0,St. James' Park,Martin Atkinson,Match Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,E0,2022-03-13,West Ham,Aston Villa,2,1,H,0,0,D,...,14:00 (15:00),West Ham,1.2,2–1,1.1,Aston Villa,59957.0,London Stadium,Jarred Gillett,Match Report
1419,E0,2022-03-13,Arsenal,Leicester City,2,0,H,1,0,H,...,16:30 (17:30),Arsenal,2.3,2–0,0.3,Leicester City,60111.0,Emirates Stadium,Anthony Taylor,Match Report
1420,E0,2022-03-14,Crystal Palace,Manchester City,0,0,D,0,0,D,...,20:00 (21:00),Crystal Palace,0.8,0–0,2.3,Manchester City,25309.0,Selhurst Park,Martin Atkinson,Match Report
1421,E0,2022-03-16,Brighton,Tottenham,0,2,A,0,1,A,...,19:30 (20:30),Brighton,0.7,0–2,2.4,Tottenham,NaN,The American Express Community Stadium,Robert Jones,Match Report


In [71]:
#Load Fifa Ratings
teams_fifa_21 = pd.read_excel('FIFA_2020_Team_Ratings.xlsx')
teams_fifa_21.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle Utd', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester United', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)
teams_fifa_21['HomeTeam'] = teams_fifa_21['Squad']
teams_fifa_21['AwayTeam'] = teams_fifa_21['Squad']
teams_fifa_21['Rating_Home'] = teams_fifa_21['Ovr']
teams_fifa_21['Rating_Away'] = teams_fifa_21['Ovr']
teams_fifa_21

,Rk,Squad,Att,Mid,Def,Ovr,HomeTeam,AwayTeam,Rating_Home,Rating_Away
0,1,Manchester City,86,85,84,85,Manchester City,Manchester City,85,85
1,2,Manchester Utd,84,81,81,82,Manchester Utd,Manchester Utd,82,82
2,3,Liverpool,89,83,86,85,Liverpool,Liverpool,85,85
3,4,Chelsea,79,82,81,81,Chelsea,Chelsea,81,81
4,5,Leicester City,79,78,81,80,Leicester City,Leicester City,80,80
5,6,West Ham United,79,78,76,78,West Ham United,West Ham United,78,78
6,7,Tottenham,88,81,81,82,Tottenham,Tottenham,82,82
7,8,Arsenal,82,81,78,80,Arsenal,Arsenal,80,80
8,9,Leeds United,71,73,72,72,Leeds United,Leeds United,72,72
9,10,Everton,80,75,79,78,Everton,Everton,78,78


In [72]:
#transform FIFA Ratings for merge
fifa_rating_home = pd.DataFrame(teams_fifa_21, columns=['HomeTeam', 'Rating_Home'])
fifa_rating_away = pd.DataFrame(teams_fifa_21, columns=['AwayTeam', 'Rating_Away'])

In [73]:
#merge dataframes
PL_20_21['Date'] = pd.to_datetime(PL_20_21['Date'], dayfirst=True)

PL_21_team_ratings = pd.merge(PL_20_21, fifa_rating_home, on='HomeTeam', how='left')
PL_21_team_ratings = pd.merge(PL_21_team_ratings, fifa_rating_away, on='AwayTeam', how='left')

PL_21_team_ratings['Rating difference'] = PL_21_team_ratings['Rating_Home']-PL_21_team_ratings['Rating_Away']

PL_21_team_ratings

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Rating_Home,Rating_Away,Rating difference
0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.89,2.02,1.91,2.13,1.92,2.02,1.87,73,80,-7
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,2.13,1.79,2.17,1.85,2.18,1.79,2.12,77,76,1
2,E0,2020-09-12,17:30,Liverpool,Leeds United,4,3,H,3,2,...,2.05,1.85,2.08,1.90,2.16,1.84,2.04,85,72,13
3,E0,2020-09-12,20:00,West Ham United,Newcastle United,0,2,A,0,0,...,1.87,2.04,1.88,2.09,1.91,2.02,1.86,78,77,1
4,E0,2020-09-13,14:00,West Bromwich Albion,Leicester City,0,3,A,0,0,...,1.98,1.93,1.99,1.95,2.01,1.91,1.97,72,80,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,2,0,H,1,0,...,2.04,1.88,2.03,1.98,2.14,1.88,2.00,85,77,8
376,E0,2021-05-23,16:00,Manchester City,Everton,5,0,H,2,0,...,1.89,1.99,1.89,2.20,2.00,2.03,1.85,85,78,7
377,E0,2021-05-23,16:00,Sheffield United,Burnley,1,0,H,1,0,...,1.86,2.05,1.86,2.17,1.90,2.03,1.84,75,76,-1
378,E0,2021-05-23,16:00,West Ham United,Southampton,3,0,H,2,0,...,1.90,2.02,1.91,2.06,2.01,1.99,1.89,78,76,2


In [75]:
#export Data to csv
PL_21_team_ratings.to_csv('Data/2020-21/Fixtures/PL20_21_fixtures.csv',index=False)